# Purpose: To Test the Sci-kit Image Thresholds for the Phuong_BEV Collaboration Images

Date Created: October 12th, 2021

Dates Edited: October 12th, 2021 - Converted from the original notebook for the microFIBER paper
Edited: April 13th, 2022 - Changed some markdown documentation stating the wrong image set

October 27th, 2021 - Downloaded all the .tiff files and changed the imports

*Step 1: Import necessary packages*

In [ ]:
#import operating system
import os

# import major packages
import numpy as np
import matplotlib.pyplot as plt
import skimage
import PIL as Image

Separating the two imports so that it is a cleaner workflow

In [ ]:
# import specific package functions
from skimage import io
from skimage.filters import try_all_threshold
from skimage.filters import threshold_isodata
from skimage.filters import threshold_li
from skimage.filters import threshold_mean
from skimage.filters import threshold_minimum
from skimage.filters import threshold_otsu
from skimage.filters import threshold_triangle
from skimage.filters import threshold_yen

from skimage import morphology
from scipy import ndimage
from skimage.measure import label

## Phuong's Data Set

*Step 2: Import image for testing thresholds*

In [ ]:
#Defining the local folder location for all files
im_folder_location = '/Users/hhelmbre/Desktop/Phuong_Bev'

In [ ]:
def folder_cleaner(folder, image_type):
    k=0
    for files in folder:
        if image_type in str(files):
            k+=1
        else:
            folder = np.delete(folder, np.argwhere(folder == str(files)))
    return folder

In [ ]:
#Getting a list of all the seprate animal model folders
arr = os.listdir(im_folder_location)
animal_list = np.asarray(arr)

#Since I am working on a mac operating system, this removes the DS_store file
animal_list = animal_list[1:]
animal_list

In [ ]:
#Getting a list of all the images we want to run and test thresholds on
try_all_im_list = []
for folders in animal_list:
    sub_folder_location = str(im_folder_location + '/' + folders)
    arr = os.listdir(sub_folder_location)
    image_list = np.asarray(arr)
    image_list = [ x for x in image_list if "TILE" not in x ]
    image_list = [ x for x in image_list if "all_thresh" not in x ]
    image_list = [x for x in image_list if ".tif" in x]
    for images in image_list:
        file_name = str(im_folder_location + '/' + folders + '/' + images)
        try_all_im_list.append(file_name)

try_all_im_list

In [ ]:
#Testing which channel the iba stain is on
im = io.imread(try_all_im_list[0])
green_im = im[:,:,1]
plt.imshow(green_im, cmap='Greys_r')

*Step 3: Trying multiple tresholds at once and saving the composite image*

In [ ]:
j = 0 #initialize a counter
for images in try_all_im_list:
    im_name = try_all_im_list[j]
    im = io.imread(im_name)
    microglia_im = im[:,:,1]
    fig, ax = try_all_threshold(microglia_im, figsize=(10, 8), verbose=False)

    
    j += 1
    fig_name = str(im_name[:-4] + '_all_thresh.tif')
    fig.savefig(fig_name)
    plt.close('all')
    print(j)

*Move all images with try_all_thresh into their own folder*

In [ ]:
import shutil
import os

source_dir = '/Users/hhelmbre/Desktop/Phuong_Bev/Healthy_OGD_Controls_tif'
target_dir = '/Users/hhelmbre/Desktop/Phuong_Bev/try_all_thresh'

arr = os.listdir(source_dir)
file_list = np.asarray(arr)

file_list = [x for x in file_list if "all_thresh" in x]

    
for file_name in file_list:
    shutil.move(os.path.join(source_dir, file_name), target_dir)

*Step 4: Based on visual inspection of the try all thresholds, Li threshold does the best*

To determine the size of the small object we should remove we took the size of microglia which are approximately 1600 um^2 according to (https://jneuroinflammation.biomedcentral.com/articles/10.1186/s12974-014-0182-7)

The confocal settings for the OGD severity study provided a 1 pixel = 3.4527 um conversion. To ensure we didn't cut off any potential microglia, we chose a cutoff on the small end to be 1/2 of the average which converts to 71 pixels ^2 used in the min_size of the remove small objects function from sci-kit image.

In [ ]:
im_folder_location = '/Users/hhelmbre/Desktop/Phuong_Bev'

#Getting a list of all the seprate animal model folders
arr = os.listdir(im_folder_location)
animal_list = np.asarray(arr)

#Since I am working on a mac operating system, this removes the DS_store file
animal_list = animal_list[1:]
animal_list = folder_cleaner(animal_list, 'tif')

#Getting a list of all the images we want to run and test thresholds on
try_all_im_list = []
for folders in animal_list:
    sub_folder_location = str(im_folder_location + '/' + folders)
    arr = os.listdir(sub_folder_location)
    image_list = np.asarray(arr)
    image_list = [ x for x in image_list if "TILE" not in x ]
    image_list = [ x for x in image_list if "all_thresh" not in x ]
    image_list = [x for x in image_list if ".tif" in x]
    for images in image_list:
        file_name = str(im_folder_location + '/' + folders + '/' + images)
        try_all_im_list.append(file_name)

In [ ]:
#li threshold
k = 0
source_dir = '/Users/hhelmbre/Desktop/Phuong_Bev/'

for images in try_all_im_list:
    im_name = try_all_im_list[k]
    im = io.imread(im_name)
    microglia_im = im[:,:,1]
    thresh_li = skimage.filters.threshold_li(microglia_im)
    binary_li = microglia_im > thresh_li
    new_binary_li = morphology.remove_small_objects(binary_li, min_size=71)
    new_binary_li = ndimage.binary_fill_holes(new_binary_li)
    
    short_im_name = im_name.split('/')[6]

    np.save(str(source_dir + 'li_thresh/' + short_im_name[:-4] + '_li_thresh'), new_binary_li)
    
    print(k)
    k += 1